In [27]:
import pymysql
import pandas as pd
from IPython.display import display, clear_output
from ipywidgets import Dropdown
connect = pymysql.connect(host='127.0.0.1', user='root', password='0000', db='practice', charset='utf8')
cur = connect.cursor()

test = "SELECT * FROM practice.m_area;"
test1 = "SELECT * FROM practice.m_store;"
query1 = "SELECT * FROM practice.tbl_order_202105;"

cur.execute(test)
test = pd.DataFrame(cur.fetchall())
test.columns = ['area_cd', 'wide_area', 'narrow_area']

cur.execute(test1)
test1 = pd.DataFrame(cur.fetchall())
test1.columns = ['store_id', 'store_name', 'area_cd']


cur.execute(query1)
data1 = pd.DataFrame(cur.fetchall())
data1.columns = ['order_id', 'store_id', 'customer_id', 'coupon_cd', 'sales_detail_id',
                 'order_accept_date', 'delivered_date', 'takeout_flag', 'total_amount', 'status']


In [44]:
import glob
import os

current_dir = os.getcwd()
tbl_order_file = os.path.join(current_dir,'tbl_order_*.csv')
tbl_order_files = glob.glob(tbl_order_file)

order_all = pd.DataFrame()
for file in tbl_order_files:
    order_tmp = pd.read_csv(file)
    print(f'{file}:{len(order_tmp)}')
    order_all = pd.concat([data1,order_tmp],ignore_index=True)

order_all = data1.loc[data1['store_id']!=999]

order_all = pd.merge(data1,test1,on='store_id',how='left')
order_all = pd.merge(order_all,test,on='area_cd',how='left')

order_all.loc[order_all['takeout_flag']==0,'takeout_name'] = 'delivery'
order_all.loc[order_all['takeout_flag']==1,'takeout_name'] = 'takeout'

order_all.loc[order_all['status']==0,'status_name'] = '주문 접수'
order_all.loc[order_all['status']==1,'status_name'] = '결제 완료'
order_all.loc[order_all['status']==2,'status_name'] = '배달 완료'
order_all.loc[order_all['status']==9,'status_name'] = '주문 취소'

order_all.loc[:,'order_date'] = pd.to_datetime(
    order_all['order_accept_date']).dt.date

order_all.groupby(['store_id','customer_id'])['total_amount'].describe()

count          mean           std      min      25%   
store_id customer_id                                                        
1        C00244531      4.0  28185.000000   7397.920879  22520.0  24162.5  \
         C00493736      1.0  26470.000000           NaN  26470.0  26470.0   
         C01249550      3.0  19493.333333   1052.916584  18570.0  18920.0   
         C02241044      5.0  37246.000000  13099.495410  22520.0  23280.0   
         C02859946      4.0  28960.000000  11596.715052  18570.0  19095.0   
...                     ...           ...           ...      ...      ...   
999      C98965135      5.0  33730.000000   8357.323136  19000.0  35820.0   
         C99065779      6.0  28203.333333  12400.684927   9390.0  21847.5   
         C99196263      6.0  32583.333333  10005.827635  23280.0  26872.5   
         C99428523      5.0  30050.000000  11522.768764  19000.0  21120.0   
         C99813215      2.0  31570.000000   6010.407640  27320.0  29445.0   

                          50%      75%      max  
store_id customer_id                             
1        C00244531    25605.0  29627.5  39010.0  
         C00493736    26470.0  26470.0  26470.0  
         C01249550    19270.0  19955.0  20640.0  
         C02241044    46590.0  46920.0  46920.0  
         C02859946    29135.0  39000.0  39000.0  
...                       ...      ...      ...  
999      C98965135    36170.0  38650.0  39010.0  
         C99065779    30460.0  34100.0  44620.0  
         C99196263    29290.0  34752.5  51000.0  
         C99428523    26470.0  37420.0  46240.0  
         C99813215    31570.0  33695.0  35820.0  

[41022 rows x 8 columns]